%matplotlib inline

In [1]:
print("QQ")

QQ


In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import time

In [4]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name

log_file_name='01_23_10:03:15.log'


In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [6]:
eval_df = pd.read_csv('../../data/Manually_Annotated_file_lists/validation_face_mesh_crop.csv')
eval_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + eval_df.subDirectory_filePath


eval_df = eval_df[eval_df['have_facemesh']]

In [7]:
batch_size = 8
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()
eval_logs = []

In [8]:
import os
model_files = os.listdir("./models")
model_files.remove('.ipynb_checkpoints')

In [9]:
len(model_files)

144

In [10]:
model_files.sort()

In [11]:
eval_result_df = pd.read_csv('./eval_df.csv')

In [12]:
model_files = list(filter(lambda x: x not in list(eval_result_df.model), model_files))

In [13]:
model_files

['01_22_23:41:48_train_mobilenet_from_pretrain_v3_add_valence_arousal_epoch3_batch_8',
 '01_22_23:43:17_mobilenet_from_pretrain_v3_epoch6_batch_32',
 '01_22_23:44:07_train_mobilenet_from_pretrain_v3_all_data_epoch2_batch_32',
 '01_22_23:58:55_train_mobilenet_from_pretrain_v3_norm_val_aro_epoch5_batch_32',
 '01_22_23:59:34_train_mobilenet_from_pretrain_v3_norm_val_aro_all_data_epoch2_batch_32']

In [14]:
for model_file in model_files:
    print(model_file)
    model = tf.keras.models.load_model('./models/'+model_file)

    total_process = 0
    start_time = time.time()

    total_loss = 0
    total_step = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_emotion_correct = 0
    for step, row in tqdm.tqdm(enumerate(DataFrameBatchIterator(eval_df, batch_size=batch_size)), total = len(eval_df) / batch_size):
        if len(row) == 0:
            continue
        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))

        img_array = np.array(list(imgs))

        
        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)


        logits = model(img_array, training=True)

        if model.output_shape != (None, 11):
            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]
            
            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
        else:
            valence_loss = 0
            arousal_loss = 0
            pred_emotion = logits
            
        emotion_loss = SCC_loss(y_emotion, pred_emotion)

        loss = valence_loss + arousal_loss + emotion_loss

        total_loss += float(loss)
        total_step += 1
        total_process += len(img_array)

        total_valence_loss += float(valence_loss)
        total_arousal_loss += float(arousal_loss)
        total_emotion_loss += float(emotion_loss)

        emotion_correct = pred_emotion.numpy().argmax(axis = 1) == y_emotion
        total_emotion_correct += sum(emotion_correct)

    #lprint(f"--- {time.time() - start_time} seconds for evaluate {total_process} image ---\n")
    eval_logs.append({
        'model': model_file,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    })

01_22_23:41:48_train_mobilenet_from_pretrain_v3_add_valence_arousal_epoch3_batch_8


682it [01:01, 11.08it/s]                                                        


01_22_23:43:17_mobilenet_from_pretrain_v3_epoch6_batch_32


682it [00:59, 11.44it/s]                                                        


01_22_23:44:07_train_mobilenet_from_pretrain_v3_all_data_epoch2_batch_32


682it [00:59, 11.42it/s]                                                        


01_22_23:58:55_train_mobilenet_from_pretrain_v3_norm_val_aro_epoch5_batch_32


682it [00:59, 11.54it/s]                                                        


01_22_23:59:34_train_mobilenet_from_pretrain_v3_norm_val_aro_all_data_epoch2_batch_32


682it [00:59, 11.47it/s]                                                        


In [15]:
df = pd.DataFrame(eval_logs)

In [16]:
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [17]:
df

,model,total_emotion_correct,total_loss,total_valence_loss,total_arousal_loss,total_emotion_loss,total_process,time_take
0,01_22_23:41:48_train_mobilenet_from_pretrain_v3_add_valence_arousal_epoch3_batch_8,209,NaN,486.207897,666.661965,NaN,5452,61.564117
1,01_22_23:43:17_mobilenet_from_pretrain_v3_epoch6_batch_32,2133,2638.735483,472.577637,636.375360,1529.782493,5452,59.622499
2,01_22_23:44:07_train_mobilenet_from_pretrain_v3_all_data_epoch2_batch_32,1780,5301.993122,520.698058,616.628619,4164.666450,5452,59.758858
3,01_22_23:58:55_train_mobilenet_from_pretrain_v3_norm_val_aro_epoch5_batch_32,2114,2646.462537,536.713889,618.067169,1491.681482,5452,59.091290
4,01_22_23:59:34_train_mobilenet_from_pretrain_v3_norm_val_aro_all_data_epoch2_batch_32,1801,5575.935055,588.779743,621.969420,4365.185887,5452,59.489157


In [18]:
eval_result_df = eval_result_df.append(df)

In [19]:
eval_result_df.to_csv('./eval_df.csv', index_label=False)

In [20]:
eval_result_df.shape

(146, 8)

In [21]:
eval_result_df.sort_values('total_valence_loss')

,model,total_emotion_correct,total_loss,total_valence_loss,total_arousal_loss,total_emotion_loss,total_process,time_take
31,01_20_22:19:05_mobilenet_from_pretrain_emotion_epoch14_batch_32,2081,2162.725327,0.000000,0.000000,2162.725327,5452,40.080483
34,01_20_22:19:49_mobilenet_from_pretrain_emotion_freeze_base_epoch12_batch_32,2094,1437.168509,0.000000,0.000000,1437.168509,5452,35.800325
33,01_20_22:19:49_mobilenet_from_pretrain_emotion_freeze_base_epoch11_batch_32,2082,1423.731995,0.000000,0.000000,1423.731995,5452,35.951856
32,01_20_22:19:05_mobilenet_from_pretrain_emotion_epoch15_batch_32,2077,2320.927839,0.000000,0.000000,2320.927839,5452,39.905126
17,01_20_22:19:49_mobilenet_from_pretrain_emotion_freeze_base_epoch6_batch_32,2111,1378.440234,0.000000,0.000000,1378.440234,5452,96.442421
30,01_20_22:19:05_mobilenet_from_pretrain_emotion_epoch13_batch_32,2101,2157.608396,0.000000,0.000000,2157.608396,5452,40.069727
29,01_20_22:19:05_mobilenet_from_pretrain_emotion_epoch12_batch_32,2097,2013.579956,0.000000,0.000000,2013.579956,5452,40.205982
28,01_20_22:19:05_mobilenet_from_pretrain_emotion_epoch11_batch_32,2092,1947.968161,0.000000,0.000000,1947.968161,5452,40.075981
16,01_20_22:19:49_mobilenet_from_pretrain_emotion_freeze_base_epoch5_batch_32,2094,1371.410057,0.000000,0.000000,1371.410057,5452,97.651011
5,01_20_22:19:49_mobilenet_from_pretrain_emotion_freeze_base_epoch10_batch_32,2105,1413.763435,0.000000,0.000000,1413.763435,5452,96.825620
